## import modules

In [4]:
from tqdm import tqdm
import re
import torch
import os

In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # 디버깅 위한 세팅
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print('Device:', torch.cuda.device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   
print('Current cuda device:', torch.cuda.current_device()) 

Device: <class 'torch.cuda.device'>
Count of using GPUs: 2
Current cuda device: 0


### Load Data
Data가 위치한 PATH에서 Data를 불러오기

In [7]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [8]:
train_data = jsonlload('nikluge-sa-2022-train.jsonl')
dev_data = jsonlload('nikluge-sa-2022-dev.jsonl')

## Preprocessing

In [9]:
def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|]+", "", text)

    # 연속 공백 제거
    text = re.sub(" +", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()